# Jupyter Notebook Script for Predicting Quantities

## Import Libraries


In [2]:
# Oudere libraries importeren
import pandas as pd
import numpy as np
import pyodbc
import sqlite3

# Nieuwe libraries importeren
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from db_utils.db_utils import create_connection, run_query, create_local_connection, run_local_query

## Load Data

Hieronder wordt aangenomen dat de gegevens al zijn ingeladen en verwerkt zoals nodig
Indien nodig kunnen we de code toevoegen voor het inlezen en voorbewerken van de gegevens

In [3]:
transaction_history_df = run_query("SELECT * FROM PayHistory")

transaction_history_df

c:\Users\lfatm\Documents\School\deds_proj_groep1\db_utils\db_utils.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,IDSK,TransactionID,ProductID,ReferenceOrderID,ReferenceOrderLineID,TransactionDate,TransactionType,Quantity,ActualCost,ModifiedDate
0,1,100000,784,41590,0,2013-07-31,W,2,0.00,2013-07-31
1,2,100001,794,41591,0,2013-07-31,W,1,0.00,2013-07-31
2,3,100002,797,41592,0,2013-07-31,W,1,0.00,2013-07-31
3,4,100003,798,41593,0,2013-07-31,W,1,0.00,2013-07-31
4,5,100004,799,41594,0,2013-07-31,W,1,0.00,2013-07-31
...,...,...,...,...,...,...,...,...,...,...
202691,89249,89249,870,53448,3,2013-07-30,S,1,4.99,2013-07-30
202692,89250,89250,872,53448,2,2013-07-30,S,1,8.99,2013-07-30
202693,89251,89251,977,53448,1,2013-07-30,S,1,539.99,2013-07-30
202694,89252,89252,711,53449,2,2013-07-30,S,1,34.99,2013-07-30


## Data Preprocessing

### Data splitsen om transacties te maken

In [4]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Convert the dataframe to a transactional format
transactions = transaction_history_df.groupby(['TransactionID', 'ProductID'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('TransactionID')

# # Apply the Apriori algorithm to find frequent itemsets
# frequent_itemsets = apriori(transactions, min_support=0.1, use_colnames=True)

# # Generate association rules from the frequent itemsets
# rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# # Print the frequent itemsets and association rules
# print("Frequent Itemsets:")
# print(frequent_itemsets)
# print("\nAssociation Rules:")
# print(rules)

# Convert the DataFrame to a list of lists
transactions_list = transactions.apply(lambda row: row[row > 0].index.tolist(), axis=1).tolist()

In [5]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions_list).transform(transactions_list)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [6]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.1, use_colnames=True)

In [7]:
from mlxtend.frequent_patterns import association_rules

# Frequent itemsets genereren met een lage min_support, deze waarde kan gewijzigd worden
frequent_itemsets = apriori(df, min_support=0.005, use_colnames=True)

# Association rules genereren met behulp van de frequent itemsets en een minimum threshold voor de lift
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.1)

# Regels sorteren op basis van confidence en lift
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

# Bovenste regels weergeven
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


In [8]:
import seaborn as sns

# Data weergeven met behulp van een scatterplot
plt.figure(figsize=(10, 6))
plt.scatterplot(x="support", y="confidence", size="lift", data=rules)
plt.xlabel("Support")
plt.ylabel("Confidence")
plt.title("Association Rules")
plt.show()

c:\Users\lfatm\AppData\Local\pypoetry\Cache\virtualenvs\deds-proj-groep1-beKqzd2--py3.12\Lib\site-packages\IPython\core\pylabtools.py:77: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
  warnings.warn(
c:\Users\lfatm\AppData\Local\pypoetry\Cache\virtualenvs\deds-proj-groep1-beKqzd2--py3.12\Lib\site-packages\IPython\core\pylabtools.py:77: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
  warnings.warn(
c:\Users\lfatm\AppData\Local\pypoetry\Cache\virtualenvs\deds-proj-groep1-beKqzd2--py3.12\Lib\site-packages\IPython\core\pylabtools.py:77: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.
  warnings.warn(


AttributeError: module 'matplotlib.pyplot' has no attribute 'scatterplot'

<Figure size 1000x600 with 0 Axes>